In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import Normalizer
from scipy.special import expit # Optimized sigmoid computation

# Auto-reload imports
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
# Get the data from the UCI archives
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data'
data = pd.read_csv(url, header=None)

# Extract features and spam classification from dataframe
X, y = data.iloc[:, :-1].values, data.iloc[:, -1].values

# Save 20% for testing the finished model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Since train_test_split randomized, drop the last 10% of training data for cross_validation
split = int(0.90 * X_train.shape[0])
X_train, X_val = X_train[:split], X_train[split:]
y_train, y_val = y_train[:split], y_train[split:]

nm = Normalizer('l2')
X_train_norm = nm.fit_transform(X_train)

# Center and normalize the data
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std
X_val = (X_val - mean) / std

print(np.max(X_train_norm, axis=0) - np.min(X_train_norm, axis=0))

[ 0.27299717  0.80694916  0.47809144  0.14567319  0.84806691  0.44178221
  0.1602196   0.64611951  0.14825776  0.87767941  0.06802817  0.44049872
  0.39804894  0.44671561  0.06451953  0.6772246   0.23810535  0.87558336
  0.96555755  0.85068294  0.64611951  0.05543338  0.14816806  0.54643033
  0.98532928  0.9122017   0.99508548  0.24644833  0.65089052  0.31863449
  0.12843525  0.12843525  0.50980222  0.12843525  0.84876158  0.12843525
  0.2062547   0.56920542  0.56361944  0.12843525  0.3799113   0.95858704
  0.20755209  0.95152537  0.96786784  0.90383997  0.14777225  0.53896806
  0.11105125  0.56970581  0.7760384   0.94534807  0.12564934  0.15661777
  0.57719984  0.69710745  0.91043282]


In [3]:
def eval_numerical_gradient(f, x, verbose=True, h=0.00001):
    """ 
    a naive implementation of numerical gradient of f at x 
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """ 

    fx = f(x) # evaluate function value at original point
    grad = np.zeros_like(x)
    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h # increment by h
        fxph = f(x) # evalute f(x + h)
        x[ix] = oldval - h
        fxmh = f(x) # evaluate f(x - h)
        x[ix] = oldval # restore

        # compute the partial derivative with centered formula
        grad[ix] = (fxph - fxmh) / (2 * h) # the slope
        if verbose:
            print(ix, grad[ix])
        it.iternext() # step to next dimension

    return grad

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [42]:
# Build our MLP class
class mlp(object):
    """
    Multi-layer perceptron class, implements a simple two layer neural network for binary classification
    using ReLU activation for the hidden layer
    """
    def __init__(self, n_features, n_hidden, std=1e-3):
        """
        Inputs:
        -------
            n_features (int): number of features in input data
            n_hidden (int): number of neurons (units) in hidden layer
            std (float): multiplier for randomly initialized weight matrices
        """
        self.params = {}
        self.params['W1'] = np.random.randn(n_features, n_hidden) * std
        self.params['W2'] = np.random.randn(n_hidden, 1) * std
        self.params['b1'] = np.zeros(n_hidden)
        self.params['b2'] = np.zeros((1, 1))
        
    def loss(self, X, y=None, reg=0.01):
        """
        Inputs:
        -------
        X (n-dimensional array): Input data
        y (n-dimensional array): Output data (classifier labels)
        
        Outputs:
        --------
        If y is None, output the class scores
        If y is given, output the loss and gradient WRT self.params
        """
        
        loss = 0.0
        grads = {}
        for key in self.params:
            # Initialize all gradient matrices to zero
            grads[key] = np.zeros(self.params[key].shape)
            
        # Compute the feed-forward result of the network
        z1 = np.dot(X, self.params['W1']) + self.params['b1'] # Samples x hidden count
        a2 = np.maximum(0, z1) # Samples x hidden count
        z3 = np.dot(a2, self.params['W2']) + self.params['b2'] # Samples x 1
        a3 = expit(z3) # Samples x 1
        scores = z3
        
        # If y was not passed, we are done
        if y is None:
            return scores
        
        loss = np.mean(y * np.log(a3) + (1 - y) * np.log(1 - a3))
        loss -= 0.5 * reg * np.sum(self.params['W1'] * self.params['W1'])
        loss -= 0.5 * reg * np.sum(self.params['W2'] * self.params['W2'])
        
        # Backpropagate the loss through the network
        dscores = (y - a3) / X.shape[0] #/ X.shape[0] # Samples x 1
        grads['W2'] = np.dot(a2.T, dscores)
        grads['b2'] = np.sum(dscores, axis=0)
        d_a2 = np.dot(dscores, self.params['W2'].T) # Samples x hidden count
        d_z1 = d_a2 * (a2 > 0) # Samples x hidden count
        grads['W1'] = np.dot(X.T, d_z1) # Features x hidden count
        grads['b1'] = np.sum(d_z1, axis=0)
        
        grads['W2'] -= reg * self.params['W2']
        grads['W1'] -= reg * self.params['W1']
        
        return loss, grads
        
    def predict(self, X):
        scores = self.loss(X)
        pred = scores > 0
        return pred
    
    def train(self, X, y, learning_rate=1e-3, 
              nb_epoch=10, batch_size=512, reg=0.1,
              learning_rate_decay=0.99):
        X_batches, y_batches = [], []
        for i in range(int(X.shape[0] / batch_size)):
            X_batches.append(X[i * batch_size : (i+1) * batch_size, :])
            y_batches.append(y[i * batch_size : (i+1) * batch_size])
            
        for epoch in range(nb_epoch):
            for X_batch, y_batch in zip(X_batches, y_batches):
                loss, grads = self.loss(X_batch, y_batch, reg)
                
                # Adding the gradient might seem unusual, until we observe
                # that the loss function takes the logarithm of a value less
                # than zero, which always yields negative loss, meaning we
                # are actually maximizing the loss during backpropagation
                self.params['W1'] += learning_rate * grads['W1']
                self.params['W2'] += learning_rate * grads['W2']
                self.params['b1'] += learning_rate * grads['b1']
                self.params['b2'] += learning_rate * grads['b2']
            y_pred = self.predict(X_train)
            acc = np.mean(y_train == y_pred)
            print('\rEpoch %d accuracy: %.2f' % (epoch, acc * 100), end='')
            learning_rate *= learning_rate_decay
                

In [39]:
a = np.random.randn(10, 4)
b = np.random.randint(1, 10)

net = mlp(a.shape[1], 5, np.sqrt(2 / a.shape[0]))
loss, grads = net.loss(a, b, reg=0.1)

for param_name in grads:
    f = lambda W: net.loss(a, b, reg=0.1)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

W2 max relative error: 1.552781e-11
b2 max relative error: 2.122749e-12
W1 max relative error: 1.442160e-10
b1 max relative error: 5.841179e-11


In [43]:
net = mlp(X_train_norm.shape[1], 1000, std=np.sqrt(2 / X.shape[0]))
np.mean(net.predict(X_train_norm) == y_train)

0.39930555555555558

In [44]:
net.train(X_train_norm, y_train[:, np.newaxis], nb_epoch=100, 
          learning_rate=5e-3, learning_rate_decay=0.999, reg=0.1)

Epoch 99 accuracy: 60.51